https://github.com/veribilimiokulu/udemy-apache-spark/blob/master/kurs_esnasi/pyspark/MachineLearning/Regression/BasitLineerRegresyon.ipynb

###Yalnızca korelasyon katsayısı pozitif veya negatif 0,50 veya ötesindeyse regresyon analizi yapılır.###

In [1]:
!pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext


sc = SparkContext('local')
spark = SparkSession(sc)
sc

<SparkContext master=local appName=pyspark-shell>

In [2]:
df =spark.read.csv("advertising.csv", header=True , inferSchema = True, sep = "," )

df.toPandas().head ()

In [3]:
#Burada çoklu değil, basit lineer regresyon olduğu için TV-Radio-Newspaper adlı 3 sütundaki sayılar toplandı
# basit lineer regresyon sadece 1 baımlı ve 1 bağımsız değişkene sahip veri setleriyle yapılır.
# ve bü üç sütunun sayı değerlerinin toplamının sonucu "Sales" adında yeni bir sütunda toplandı
# eski 3 sütun sonunda silindi

df2 = df.withColumn("Advertisement", df.TV + df.Radio + df.Newspaper) \
.withColumnRenamed("Sales","label") \
.drop("TV","Radio","Newspaper")

df2.toPandas().head()

,ID,label,Advertisement
0,1,22.1,337.1
1,2,10.4,128.9
2,3,9.3,132.4
3,4,18.5,251.3
4,5,12.9,250.0


## Veri hazırlığı ve Keşfi

In [4]:
df2.describe("label","Advertisement").toPandas().head()

,summary,label,Advertisement
0,count,200,200
1,mean,14.022500000000003,200.86049999999992
2,stddev,5.217456565710477,92.9851805869837
3,min,1.6,11.7
4,max,27.0,433.6


In [5]:
# 200 adet verinin hepsi gözüküyor,demek ki hiç null veri yok
# kategorik değişken olmadığı için stringindexer ve one hot encoder  kullanmayacağız 


## VectorAssembler

In [6]:
from pyspark.ml.feature import VectorAssembler
vector_assembler = VectorAssembler() \
.setInputCols(["Advertisement"]) \
.setOutputCol("features")

## Regresyon Modeli

In [7]:
from pyspark.ml.regression import LinearRegression
linear_reg_obj = LinearRegression() \
.setLabelCol("label")\
.setFeaturesCol("features")

##  pipeline oluşturuyoruz

In [8]:
from pyspark.ml import Pipeline
pipeline_obj = Pipeline() \
.setStages([vector_assembler, linear_reg_obj])

## veri setini test ve train olarak ayırma

In [9]:
train_df, test_df = df2.randomSplit([0.8, 0.2], seed=142)


## Önceden Train olarak ayrılan set için Modeli Eğitme

In [10]:
pipeline_model = pipeline_obj.fit(train_df)

# pipeline.fit her zaman train olarak ayrılan veri setini alır,
# pipeline.transform ise her zaman değil ama genellikle test olarak ayrılan veri setini alır.
# çünkü regresyonda modelin eğitim esnasında görmediği verilerle test edilmesi daha doğrudur

## Modeli test etme

In [11]:
result_df = pipeline_model.transform(test_df)
#

 ## Sonuçlara bakma   

In [12]:
result_df.toPandas().head()

,ID,label,Advertisement,features,prediction
0,4,18.5,251.3,[251.3],16.553870
1,9,4.8,11.7,[11.7],4.929785
2,11,8.6,96.1,[96.1],9.024412
3,15,19.0,283.0,[283.0],18.091781
4,25,9.7,93.2,[93.19999999999999],8.883720


 ## Stages sorgusule Pipeline modelinin içinde hangi algoritmalar çalıştırılmış ona bakıyoruz?

In [13]:
pipeline_model.stages

[VectorAssembler_25d2d69f29b8,
 LinearRegressionModel: uid=LinearRegression_fd0f5c7c0c96, numFeatures=1]

 ## Stages sorgusu soncunda 1. indisde çıkan LİNEER REGRESYON MODELİNİN SONUÇLARINA BAKALIM

In [15]:
# ÇAĞIRMASI KOLAY OLSUN DİYE BİR NESNEYE ATADIK
lr_model = pipeline_model.stages[1]

In [16]:
#COEFFICIENT DEĞERİNİ SORGULAYALIM 
lr_model.coefficients

DenseVector([0.0485])

In [17]:
#sabiti görelim 
lr_model.intercept

4.362164413237526

In [18]:
# Toplam değişkenliği 1 olarak kabul ettiğimizde , bizim modelimiz bu değişkenliğin %75'ini açıklayabiliyormuş.
lr_model.summary.r2

0.7541832816333721

In [19]:
# Bu model anlamlı mı değil mi? p değerlerine bakalım :
lr_model.summary.pValues

[0.0, 6.661338147750939e-16]

In [20]:
#  değerine bakalım 
lr_model.summary.tValues

[22.086724008307982, 8.977906065711718]

In [21]:
# .rootMeanSquaredError , bu rakamın label (sonuç) değişkenin standart sapmasından küçük olması gerekiyor
#4. satırda standart sapma gözüküyor.
lr_model.summary.rootMeanSquaredError

2.540740128120867

In [22]:
# y = 4.537119328969264 + 0.0472 * Advertisement
# yukarıda elimzideki regresyonu hesapladık

In [23]:
# 100 bin liralık bir reklam bütçesi ile ne kadar satış gerçekleşeceğini tahmin edelim.
df_predict_rdd = spark.sparkContext.parallelize([100.0])
df_predict = df_predict_rdd.map(lambda x: (x,)).toDF(["Advertisement"])

In [24]:
df_predict.show()

+-------------+
|Advertisement|
+-------------+
|        100.0|
+-------------+



In [25]:
df_pred_vec = vector_assembler.transform(df_predict)

In [26]:
lr_model.transform(df_pred_vec).toPandas().head()

,Advertisement,features,prediction
0,100.0,[100.0],9.213619
